In [100]:
%matplotlib inline
%config InlineBackend.figure_formats = ["svg"]
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt
import aerokit.aero.Isentropic as Is
import aerokit.aero.MassFlow   as mf
from scipy.optimize import minimize

plt.rcParams["figure.dpi"] = 120
print(ct.__version__)

3.0.0


In [102]:
def objective_function(variables):
    try:
        mixture_ratio, Area_throat, pressure_exit, mdot = variables
        F = calculate_F(mixture_ratio, Area_throat, pressure_exit, mdot)
        print(F)
        return abs(F - target_F)
    except Exception as e:
        #print(f"An error occurred: {e}")
        return float('inf')  # Return a large number to signify error in calculation

In [103]:

def calculate_F(mixture_ratio, Area_throat, pressure_exit, mdot):
    pressure_atmos=101325
    molar_mass_CH4 = 16.04
    molar_mass_O2 = 32.00
    stoichiometric_ratio = molar_mass_CH4 / (2 * molar_mass_O2)

    equivalence_ratio = mixture_ratio * stoichiometric_ratio
    gas = ct.Solution('gri30.yaml')
    gox = "O2:1"
    methane = "CH4:1"

    gas.TP = 300, 179908
    gas.set_equivalence_ratio(equivalence_ratio  , methane, gox)
    gas.equilibrate("HP") # Adiabatic since holding H constant
    Adiabatic_flame_temp = gas.T
    gamma_mix = gas.cp / gas.cv
    MW_mix = gas.mean_molecular_weight
    R = 8314.5/MW_mix

    C_star = np.sqrt((R * Adiabatic_flame_temp ) / gamma_mix ) * ((gamma_mix  + 1) / 2) ** ((gamma_mix  + 1) / (2 * (gamma_mix  - 1)))
    Pc = (C_star * mdot) / Area_throat
    Mach = Is.Mach_PtPs(Pc/pressure_exit, gamma_mix)
    A_over_At = (1/Mach) * ( ( 2 + (gamma_mix-1)*Mach**2)/(gamma_mix+1))**((gamma_mix+1)/(2*(gamma_mix-1)))
    a = (2*gamma_mix **2)/(gamma_mix -1)
    b = ((2) / (gamma_mix +1))**((gamma_mix +1)/(gamma_mix -1))
    c = (1 - (pressure_exit/Pc ))**((gamma_mix -1)/gamma_mix )
    Cf  = np.sqrt(a*b*c) + (((pressure_exit/Pc ) - (pressure_atmos/Pc ))*(A_over_At ))
    F = Cf * Pc * Area_throat

    return F


In [104]:
target_F = 5000

bounds = [(0, 6),  # mixture_ratio
          (0.005, 0.065),  # Area_throat
          (50000, 110000),  # pressure_exit
          (1, 5)]  # mdot

# Initial guess 
initial_guess = [0.8, 0.005, 101325, 1.65]

result = minimize(objective_function, initial_guess, bounds=bounds, method='TNC')

print(f'Optimal Mixture Ratio {result.x[0]}')
print(f'Optimal Throat Area {result.x[1]} m^2')
print(f'Optimal Exit Pressure {result.x[2]} Pa')
print(f'Optimal Mass Flow {result.x[3]} kg/s')

3947.903684197573
3947.9037041109004
3947.903411320381
3947.9036841976217
3947.9037089511626
3947.9040570302177
3947.904076943488
3947.903784153036
3947.9040570302664
3947.9040817838077
3947.9039460408335
3947.903965954145
3947.9036731636475
3947.903946040882
3947.903970794421
6088.462650544073
6088.462678256416
6088.462407132914
6088.462650544138
6088.462675754277
5009.067116847943
5009.067140803516
5009.066862022799
5009.067116848001
5009.067141822591
4476.208090868111
4476.208112838257
4476.207828208054
4476.2080908681655
4476.208115728856
4742.065591083041
4742.065614054843
4742.065332577982
4742.065591083097
4742.065616000167
4875.423292261906
4875.423315727775
4875.423035650359
4875.423292261961
4875.423317207674
4942.209445172333
4942.20946888362
4942.209189466728
4942.209445172391
4942.209470132514
4975.629342659108
4975.629366492677
4975.629087396843
4975.629342659165
4975.629367626518
4992.345995396161
4992.346019290726
4992.345740353223
4992.345995396219
4992.3460203671875
5

In [112]:
def calculate_Chamber_Properties(mixture_ratio, Area_throat, pressure_exit, mdot):
    pressure_atmos=101325
    molar_mass_CH4 = 16.04
    molar_mass_O2 = 32.00
    stoichiometric_ratio = molar_mass_CH4 / (2 * molar_mass_O2)

    equivalence_ratio = mixture_ratio * stoichiometric_ratio
    gas = ct.Solution('gri30.yaml')
    gox = "O2:1"
    methane = "CH4:1"

    gas.TP = 300, 179908
    gas.set_equivalence_ratio(equivalence_ratio  , methane, gox)
    gas.equilibrate("HP") # Adiabatic since holding H constant
    Adiabatic_flame_temp = gas.T
    gamma_mix = gas.cp / gas.cv
    MW_mix = gas.mean_molecular_weight
    R = 8314.5/MW_mix

    C_star = np.sqrt((R * Adiabatic_flame_temp ) / gamma_mix ) * ((gamma_mix  + 1) / 2) ** ((gamma_mix  + 1) / (2 * (gamma_mix  - 1)))
    Pc = (C_star * mdot) / Area_throat
    Mach = Is.Mach_PtPs(Pc/pressure_exit, gamma_mix)
    A_over_At = (1/Mach) * ( ( 2 + (gamma_mix-1)*Mach**2)/(gamma_mix+1))**((gamma_mix+1)/(2*(gamma_mix-1)))
    a = (2*gamma_mix **2)/(gamma_mix -1)
    b = ((2) / (gamma_mix +1))**((gamma_mix +1)/(gamma_mix -1))
    c = (1 - (pressure_exit/Pc ))**((gamma_mix -1)/gamma_mix )
    Cf  = np.sqrt(a*b*c) + (((pressure_exit/Pc ) - (pressure_atmos/Pc ))*(A_over_At ))
    F = Cf * Pc * Area_throat

    return Adiabatic_flame_temp,gamma_mix,MW_mix,Pc,A_over_At,Mach,F

In [113]:
Adiabatic_flame_temp,gamma_mix,MW_mix,Pc,A_over_At,Mach,F = calculate_Chamber_Properties(result.x[0], result.x[1], result.x[2], result.x[3])

print(f'Chamber Adibatic Flame Temperature: {Adiabatic_flame_temp} K')
print(f'Mixture Gamma: {gamma_mix}')
print(f'Mixture MW: {MW_mix} g/mol')
print(f'Chamber Pressure: {Pc} Pa')
print(f'Nozzle Expansion Ratio Ae/At: {A_over_At}')
print(f'Exit Mach: {Mach}')
print(f'Total Thrust {F} N')

Chamber Adibatic Flame Temperature: 2194.602030875372 K
Mixture Gamma: 1.239860020848555
Mixture MW: 30.412381193688102 g/mol
Chamber Pressure: 484763.0496396418 Pa
Nozzle Expansion Ratio Ae/At: 1.4097265173515034
Exit Mach: 1.7161599284019347
Total Thrust 4999.999999999162 N
